
# Grid Coordinates & Rotations (Python Cheatsheet + Exercises)

**What you'll learn**
- How array indices work: `grid[row][col]` where row increases **down** and col increases **right**  
- How to convert **bottom-left origin (x→, y↑)** coordinates to array **top-left origin (row↓, col→)** indices  
- How to do **flips** (up–down, left–right), **transpose**, and **rotations** (90° CW/CCW, 180°) with simple rules  
- How to verify your code with small examples and tests



## 1. Coordinate Systems

We will use the following conventions:

- **Array view (top-left origin):** `grid[row][col]` (row increases **down**, col increases **right**).
- **Cartesian-like view (bottom-left origin):** `(x, y)` where `x` increases **right**, `y` increases **up**.

### Conversion (bottom-left → array)
For an `N × N` grid:
- **Point:**  
  \( \text{row} = N - 1 - y, \quad \text{col} = x \)
- **Rectangle (half-open) `[x1, x2) × [y1, y2)` → array `[r1, r2) × [c1, c2)`**  
  \( r_1 = N - y_2, \quad r_2 = N - y_1, \quad c_1 = x_1, \quad c_2 = x_2 \)

> **Mnemonic**: just remember **`row = N - 1 - y`** and `col = x`.



## 2. Helper Functions
Below are reusable helpers for coordinate conversion and grid transforms.


In [ ]:

from typing import List, Tuple

Grid = List[List[int]]

def xy_to_rc(N: int, x: int, y: int) -> Tuple[int, int]:
    """Convert bottom-left origin (x,y) to array indices (row, col).
    Array origin is top-left; row increases downward.
    """
    return N - 1 - y, x


def rect_xy_to_rc(N: int, x1: int, y1: int, x2: int, y2: int, half_open: bool = True) -> Tuple[int, int, int, int]:
    """Convert rectangle from bottom-left origin to array (top-left origin).
    Returns (r1, c1, r2, c2) as half-open indices suitable for slicing.
    """
    if x1 > x2:
        x1, x2 = x2, x1
    if y1 > y2:
        y1, y2 = y2, y1

    if half_open:
        r1 = N - y2
        r2 = N - y1
        c1 = x1
        c2 = x2
    else:
        # closed → convert to half-open
        r1 = N - 1 - y2
        r2 = N - y1
        c1 = x1
        c2 = x2 + 1
    return r1, c1, r2, c2


def flip_ud(A: Grid) -> Grid:
    """Flip Up–Down (reverse rows)."""
    return A[::-1]


def flip_lr(A: Grid) -> Grid:
    """Flip Left–Right (reverse cols)."""
    return [row[::-1] for row in A]


def transpose(A: Grid) -> Grid:
    """Transpose (r,c) -> (c,r)."""
    return [list(row) for row in zip(*A)]


def rot90_cw(A: Grid) -> Grid:
    """Rotate 90° clockwise.
    Trick: transpose then flip left–right.
    """
    T = transpose(A)
    return flip_lr(T)


def rot90_ccw(A: Grid) -> Grid:
    """Rotate 90° counterclockwise.
    Trick: transpose then flip up–down.
    """
    T = transpose(A)
    return flip_ud(T)


def rot180(A: Grid) -> Grid:
    """Rotate 180°. Trick: flip UD then LR (or vice versa)."""
    return flip_lr(flip_ud(A))


def make_sequential(rows: int, cols: int) -> Grid:
    """Create a rows×cols grid filled with 1..rows*cols."""
    k = 1
    g = []
    for _ in range(rows):
        row = []
        for _ in range(cols):
            row.append(k)
            k += 1
        g.append(row)
    return g


def print_grid(A: Grid) -> None:
    for row in A:
        print(" ".join(f"{v:2d}" for v in row))
    print()



## 3. Demos with a 3×3 grid


In [ ]:

A = make_sequential(3, 3)
print("Original:")
print_grid(A)

print("Flip Up–Down:")
print_grid(flip_ud(A))

print("Flip Left–Right:")
print_grid(flip_lr(A))

print("Transpose:")
print_grid(transpose(A))

print("Rotate 90° CW:")
print_grid(rot90_cw(A))

print("Rotate 90° CCW:")
print_grid(rot90_ccw(A))

print("Rotate 180°:")
print_grid(rot180(A))



## 4. Quick Tests (Assertions)
Small tests to make sure the transforms match the expected results.


In [ ]:

# Expected results for 3x3
A = [
    [1,2,3],
    [4,5,6],
    [7,8,9],
]

exp_cw = [
    [7,4,1],
    [8,5,2],
    [9,6,3],
]
exp_ccw = [
    [3,6,9],
    [2,5,8],
    [1,4,7],
]
exp_180 = [
    [9,8,7],
    [6,5,4],
    [3,2,1],
]

assert rot90_cw(A) == exp_cw, "CW rotation mismatch"
assert rot90_ccw(A) == exp_ccw, "CCW rotation mismatch"
assert rot180(A) == exp_180, "180 rotation mismatch"

# Transpose then flip relations
assert rot90_cw(A) == flip_lr(transpose(A))
assert rot90_ccw(A) == flip_ud(transpose(A))
print("All tests passed! ✅")



## 5. Coordinate Conversion Demo
Let's convert a few points/rectangles from bottom-left `(x,y)` to array `(row,col)`.


In [ ]:

N = 5
points = [(2,3), (0,0), (4,4)]
print("Point conversion (bottom-left → array):")
for (x,y) in points:
    r,c = xy_to_rc(N, x, y)
    print(f"(x,y)=({x},{y}) → (row,col)=({r},{c})")

print("\nRectangle conversion (half-open):")
rect = (1,0,4,2)  # [x1,x2)=[1,4), [y1,y2)=[0,2)
r1,c1,r2,c2 = rect_xy_to_rc(N, *rect)
print(f"[x1,x2)×[y1,y2)={rect[:2]}×{rect[2:]} → [r1,r2)×[c1,c2)=({r1},{r2})×({c1},{c2})")



## 6. Exercises (Try It Yourself)

### A. Implement `rot90_cw_alt` using only `transpose` and one flip
- Hint: rotate 90° clockwise = transpose + left–right flip


In [ ]:

def rot90_cw_alt(A: Grid) -> Grid:
    # TODO: implement using transpose() and one of the flip functions
    T = transpose(A)
    return flip_lr(T)  # <- fill-in by students; solution shown here for convenience

# quick check
A = make_sequential(3,3)
assert rot90_cw_alt(A) == rot90_cw(A)
print("rot90_cw_alt OK!")



### B. Write `rot90_ccw_alt` and `rot180_alt` using flips and/or transpose
- `rot90_ccw` = transpose + up–down flip  
- `rot180` = up–down flip + left–right flip (order doesn't matter)


In [ ]:

def rot90_ccw_alt(A: Grid) -> Grid:
    T = transpose(A)
    return flip_ud(T)

def rot180_alt(A: Grid) -> Grid:
    return flip_lr(flip_ud(A))

# quick checks
A = make_sequential(3,3)
assert rot90_ccw_alt(A) == rot90_ccw(A)
assert rot180_alt(A) == rot180(A)
print("rot90_ccw_alt / rot180_alt OK!")



## 7. Memory Tips (한 줄 요약)
- **좌하단→배열**: `row = N - 1 - y`, `col = x`  
- **회전**: **Transpose** + (CW=**flip left–right**, CCW=**flip up–down**)  
- **뒤집기**: 상하=**row만 뒤집기**, 좌우=**col만 뒤집기**  


_Generated on: 2025-09-09T11:20:11_